# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

## Open Connection

In [ ]:
import os
import re
import glob
import psycopg2
import pandas as pd
import sql_queries as sq

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
conn = psycopg2.connect(
    user="admin",
    password="<password>",
    host="localhost", 
    port="5432",
    dbname="sparkifydb"
)
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
cur = conn.cursor()
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

def single_quote_converter(sentence):
    sentence = re.sub(r'\'', r'<single_quote_tag>', sentence)

    return sentence
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#

## Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
song_files = get_files(os.path.join(os.getcwd(), "data/song_data"))

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
print(f"Total number of JSON files: {len(song_files)}")
print(f"Example index 0: {os.path.basename(song_files[0])}")

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
filepath = song_files[0]

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
songs_df = pd.read_json(filepath, lines=True)
songs_df.head()

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
print(f"Artist ID, characters lenght: {len(songs_df['artist_id'].values[0])}")
print(f"Artist ID, characters lenght: {len(songs_df['song_id'].values[0])}")

### `artists` Table

#### Extract Data for Artists Table

- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
columns = [
    "artist_id",
    "artist_name",
    "artist_location",
    "artist_latitude",
    "artist_longitude"
]
artist_data = songs_df[columns]
artist_data[:]["artist_name"] = artist_data["artist_name"].map(
    single_quote_converter, na_action='ignore'
)
artist_data[:]["artist_location"] = artist_data["artist_location"].map(
    single_quote_converter, na_action='ignore'
)
artist_data.head()
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#

#### Insert Record into Artist Table

Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query creation
query = sq.artist_table_insert(dataframe=artist_data, verbose=True)

In [ ]:
cur.execute(query)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

### `songs` Table

#### Extract Data for Songs Table

- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
columns = ["song_id", "title", "artist_id", "year", "duration"]
song_data = songs_df[columns]
song_data[:]["title"] = song_data["title"].map(
    single_quote_converter, na_action='ignore'
)
song_data.head()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query creation
query = sq.song_table_insert(dataframe=song_data, verbose=True)

In [ ]:
cur.execute(sq.song_table_insert(song_data))
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
log_files = get_files(os.path.join(os.getcwd(), "data/log_data"))

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
print(f"Total number of JSON files: {len(log_files)}")
print(f"Example index 0: {os.path.basename(log_files[0])}")

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
filepath = log_files[0]

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
logs_df = pd.read_json(filepath, lines=True)
logs_df.head()

In [ ]:
logs_df.info()

In [ ]:
logs_df["page"].value_counts()

### `time` Table

#### Extract Data for Time Table

- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
filtered_logs_df = logs_df[logs_df["page"] == 'NextSong'].copy()
filtered_logs_df.head()

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
filtered_logs_df['ts'] = pd.to_datetime(filtered_logs_df['ts'], unit='ms')
filtered_logs_df.head()

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
time_data = list(
    map(
        lambda item: [
            item.strftime('%Y-%m-%dT%H:%M:%S'),
            item.hour,
            item.day,
            item.week,
            item.month,
            item.year,
            item.weekday() + 1
        ],
        filtered_logs_df['ts']
    )
)
print(f"Print element list sample:\n{time_data[0]}\n")
column_labels = [
    'start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'
]
print(f"Print column names:\n{column_labels}\n")
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
time_list_dict = [dict(zip(column_labels, row)) for row in time_data]
print(f"Print first row dictionary sample:\n{time_list_dict[0]}\n")

In [ ]:
time_df = pd.DataFrame(time_list_dict)
print(f"Current dataframe dimensions:\n{time_df.shape}\n")
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query creation
start_index = 0
end_index = time_df.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.time_table_insert(dataframe=time_df.iloc[index:], verbose=True)
    cur.execute(query)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

### `users` Table

#### Extract Data for Users Table

- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
columns = ["userId", "firstName", "lastName", "gender", "level"]
user_df = logs_df[columns]
user_df[:]["firstName"] = user_df["firstName"].map(
    single_quote_converter, na_action='ignore'
)
user_df[:]["lastName"] = user_df["lastName"].map(
    single_quote_converter, na_action='ignore'
)
user_df[:]["gender"] = user_df["gender"].str.upper()
user_df = user_df.drop_duplicates()
user_df.head()

#### Insert Records into Users Table

Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = time_df.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.user_table_insert(dataframe=user_df.iloc[index:], verbose=True)
    cur.execute(query)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

### `songplays` Table

#### Extract Data and Songplays Table

This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## Close Connection to Sparkify Database

In [ ]:
conn.close()

> Implement `etl.py`
> Use what you've completed in this notebook to implement `etl.py`.